## Dataset & DataLoader 살펴보기

-   Pytorch에서 배치 크기만큼 데이터를 조절하기 위한 메카니즘
-   Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
-   DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달


### [1] 모듈 로딩 및 데이터 준비 <hr>


In [1]:
### 모듈 로딩
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd


### [2] 데이터셋 생성 <hr>


-   [2-1] 사용자정의 데이터셋 생성


In [2]:
### 데이터 준비
filename = "../data/text/iris.csv"

irisDF = pd.read_csv(filename)
irisDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [3]:
irisNP = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=(0, 1, 2, 3))
irisNP[:2]


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [4]:
### [2-1] 사용자정의 DataSet 클래스
# - 데이터의 Tensor 변환
class DLDataset(Dataset):
    # 초기화 콜백함수(callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data

        # ndarray ==> tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data).squeeze()
        print("[target & target SHAPE]", self.target.shape, self.target.ndim)

    # 데이터셋의 갯수 체크 콜백함수(callback function)
    def __len__(self):
        return self.target.shape[0]

    # 특정 인덱스 데이터 + 라벨 반환 콜백함수(callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]


-   [2-2] 데이터셋 인스턴스 생성


In [5]:
## 피쳐와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1]]

print(f"featureDF => {featureDF.shape}, {featureDF.ndim}D")
print(f"targetDF  => {targetDF.shape}, {targetDF.ndim}D")


featureDF => (150, 4), 2D
targetDF  => (150,), 1D


In [6]:
# object 타입 타겟 ===> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetDF)
targetNP = targetNP.reshape(-1, 1)

print(targetNP.shape, targetNP.ndim)


(150, 1) 2


In [7]:
### ===> 데이터셋 생성 -> DF, NP
my_dataset = DLDataset(featureDF, targetNP)
my_dataset[0], featureDF.iloc[0], targetDF[0]


[target & target SHAPE] torch.Size([150]) 1


((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0)),
 sepal_length    5.1
 sepal_width     3.5
 petal_length    1.4
 petal_width     0.2
 Name: 0, dtype: float64,
 'setosa')

In [8]:
# 데이터셋 생성 -> NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
my_dataset2[0]


[target & target SHAPE] torch.Size([150]) 1


(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

-   [2-3] 학습용, 검증용 테스트용 Dataset <hr>


In [9]:
### ===> 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(42)

trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator=seed)

print(
    f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개"
)

print(f"Subset 속성 =>\nindices : {trainDS.indices}\ndataset : {trainDS.dataset}")
print(f"Subset 속성 =>\nindices : {validDS.indices}\ndataset : {validDS.dataset}")


trainDS => 105개, validDS => 15개, testDS => 30개
Subset 속성 =>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset : <__main__.DLDataset object at 0x0000020C8F79D1F0>
Subset 속성 =>
indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146]
dataset : <__main__.DLDataset object at 0x0000020C8F79D1F0>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용 <hr>


In [10]:
# DataLoader 생성
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 5
trainDL = DataLoader(trainDS, batch_size=batch)
validDL = DataLoader(validDS, batch_size=batch)
testDL = DataLoader(testDS, batch_size=batch)


In [11]:
# Epoch당 반복 단위
print(f"batch_size : {batch}")
print(
    f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개"
)
print(
    f"trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개"
)


batch_size : 5
trainDS => 105개, validDS => 15개, testDS => 30개
trainDL => 21개, validDL => 3개, testDL => 6개


[4] Model 클래스 정의 : 입/출력 피쳐 수, 층 수, 은닉층의 노드수 <hr>

-   구조 설계
    -   입력측 : 입력 <= 피쳐 갯수, iris 4개
    -   은닉층 : 마음대로 알아서 잘
    -   출력측 : 출력 <= [분류] 타겟 클래스 갯수, [회귀] 1개


In [12]:
# 모델 클래스 정의
# 클래스명 : CModel
class CModel(nn.Module):
    # 구성요소 정의 함수
    def __init__(self, in_, out_):
        super().__init__()
        self.input_layer = nn.Linear(in_, 100)
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_)
        self.relu = nn.ReLU()

    # 순방향 학습 진행 함수
    def forward(self, x):
        x = self.input_layer(x)  # W1X1+W2X2+...+WnXn+b 100개 반환
        x = self.relu(x)  # relu() 결과 100개 반환
        x = self.hidden_layer(x)  # W1X1+W2X2+...+WnXn+b 27개 반환
        x = self.relu(x)  # relu() 결과 27개 반환
        x = self.output_layer(x)  # W1X1+W2X2+...+WnXn+b out_개 반환
        return x


[5] 학습 준비 : 실행디바이스, 모델, 최적화, 손실함수, 학습횟수, 학습함수, 평가함수, 예측함수 <hr>


In [22]:
# 실행 디바이스 설정
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

# 학습 횟수
EPOCHS = 5000


In [14]:
# 모델 인스턴스
IN, OUT = my_dataset2.feature.shape[1], len(torch.unique(my_dataset2.target))
model = CModel(IN, OUT).to(DEVICE)
print(f"IN: {IN}, OUT: {OUT}")
print(model)


IN: 4, OUT: 3
CModel(
  (input_layer): Linear(in_features=4, out_features=100, bias=True)
  (hidden_layer): Linear(in_features=100, out_features=27, bias=True)
  (output_layer): Linear(in_features=27, out_features=3, bias=True)
  (relu): ReLU()
)


In [15]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
OPTIMIZER = optim.Adam(model.parameters())


-   학습 및 검증관련 함수 정의


In [16]:
import torchmetrics.functional as metrics


In [17]:
### ===> 학습 진행함수
def training():
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능을 활성화
    model.train()

    # 배치크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # 배치크가만큼의 학습 데이터 준비
        feature, target = feature.to(DEVICE), target.to(DEVICE)

        # 학습
        pre_target = model(feature)
        # print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
        # print(f'target     => {target.shape}, {target.ndim}D')

        # 손실계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss.item())

        # W,b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

        # 배치 단위 학습 진행 메세지 출력
        # print(f"[Train {cnt} batch Loss] ==> {loss.item():.4f}")

    # 에포크 단위 학습 진행 메세지 출력
    acc = metrics.accuracy(pre_target, target, task="multiclass", num_classes=OUT)
    print(f"[Train loss] ==> {loss} [Train Accuracy] ==> {acc}")

    return train_loss


In [24]:
### ===> 검증 및 평가 진행함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader


def testing(dataLoader):
    # 추론모드 => 정규화, 경사하강법, 드랍아웃 등의 기능을 비활성화
    model.eval()

    with torch.no_grad():
        # 배치크기만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # 배치크가만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)

            # 학습
            pre_target = model(feature)

            # 손실계산
            loss = LOSS_FN(pre_target, target)
            val_loss.append(loss.item())

        # 에포크 단위 학습 진행 메세지 출력
        acc = metrics.accuracy(pre_target, target, task="multiclass", num_classes=OUT)
        print(f"[Valid loss] ==> {loss} [Valid Accuracy] ==> {acc}")

        return val_loss, acc.item()


[6] 학습 진행 <hr>


In [25]:
### ===> 지정된 횟수만큼 처음부터 끝까지 학습 및 검증 진행
### ===> 목표 : 최적(Error 최소화)의 W, b를 가진 모델 완성
count = 0
pre_val_loss = 0
for eps in range(EPOCHS + 1):


    # 학습
    train_loss = training()
    train_loss_mean = sum(train_loss) / len(train_loss)


    # 검증
    valid_loss, acc = testing(validDL)
    valid_loss_mean = sum(valid_loss) / len(valid_loss)

    print(f"[{eps:2d}/{EPOCHS}] TRAIN {train_loss_mean}, VALID {valid_loss_mean}\n")

    # 조기종료 기능 ==> 조건 : val_loss가 지정된 횟수(예: 5) 이상 개선이 안되면 학습 종료
    if pre_val_loss == valid_loss_mean:
        count += 1
        if count > 5:
            break
    else:
        count = 0
    pre_val_loss = valid_loss_mean


[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 2.8610207891688333e-07 [Valid Accuracy] ==> 1.0
[ 0/5000] TRAIN 2.2706529989591218e-09, VALID 9.53673596389611e-08

[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 1.4305109630186053e-07 [Valid Accuracy] ==> 1.0
[ 1/5000] TRAIN 2.2706529989591218e-09, VALID 4.768369876728684e-08

[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 2.8610207891688333e-07 [Valid Accuracy] ==> 1.0
[ 2/5000] TRAIN 2.2706529989591218e-09, VALID 9.53673596389611e-08

[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 1.4305109630186053e-07 [Valid Accuracy] ==> 1.0
[ 3/5000] TRAIN 2.2706529989591218e-09, VALID 4.768369876728684e-08

[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 2.8610207891688333e-07 [Valid Accuracy] ==> 1.0
[ 4/5000] TRAIN 2.2706529989591218e-09, VALID 9.53673596389611e-08

[Train loss] ==> 0.0 [Train Accuracy] ==> 1.0
[Valid loss] ==> 1.6689293147464923e-07 [Valid Accuracy] 

[7] 테스트 진행 <hr>


In [26]:
test_loss, acc = testing(testDL)


[Valid loss] ==> 1.5497150798182702e-06 [Valid Accuracy] ==> 1.0


In [29]:
np.mean(test_loss)


1.7105290169507725

In [28]:
acc


tensor(1.)